In [ ]:
# Need to install rpy2 and have a working version of R with LatticeKrig and dependencies installed

import rpy2
print(rpy2.__version__)
import numpy as np
import itertools

In [ ]:
from rpy2.robjects.packages import importr
# import R's "base" package
base = importr('base')

# import R's "utils" package
utils = importr('utils')

# import rpy2's package module
import rpy2.robjects.packages as rpackages

# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

packnames = ('ggplot2', 'spam64')

# R vector of strings
from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))
    
#install.packages("spam64")

import rpy2.robjects.packages as rpackages

# Install the package if not already installed
if not rpackages.isinstalled('spam64'):
    utils.install_packages('spam64')

# Load the package
spam64 = rpackages.importr('spam64')

In [ ]:
# Load the map from a single mouse data downloaded from Sit & Goard (2020)
import scipy
import numpy as np

a=scipy.io.loadmat('13476522/M5/Natural_Movies/map.mat')




In [ ]:
# Load mouse spatiotemporal data from Sit & Goard (2020) (previously converted from Matlab)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

MouseData=np.loadtxt('M5Movie.txt',delimiter=" ",skiprows=1)
plt.scatter(MouseData[:,1],MouseData[:,2],c=MouseData[:,0])


In [ ]:
# Visualise data as sanity check

import matplotlib.pyplot as plt

j=500
TempData=MouseData[:,j+3]
Coords=MouseData[:,1:3]
plt.scatter(MouseData[:,1],MouseData[:,2],c=TempData)


In [ ]:
# import numpy as np
# from rpy2.robjects.packages import importr
# from rpy2.robjects import FloatVector
# from rpy2.robjects import r, pandas2ri
# import rpy2.robjects as robjects
# import pandas as pd
# import scipy
# from sklearn.neighbors import DistanceMetric
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy import stats
# from scipy.interpolate import griddata

# pandas2ri.activate()
# LK = importr('LatticeKrig')
# tr=MouseData.shape[1]-3
# tr=540
# LKPredAllFrames=np.zeros([MouseData.shape[0],tr])
# MaskAllFrames=np.zeros([MouseData.shape[0],tr])



# # Example data

# x=MouseData[:,1]
# y=MouseData[:,2]

# # Define the grid for interpolation
# grid_x, grid_y = np.mgrid[x.min():x.max():25j, y.min():y.max():25j]

# # Perform interpolation
# Coords=MouseData[:,1:3]
# vector1 = robjects.FloatVector([Coords[:,0].min(), Coords[:,0].max()])
# vector2 = robjects.FloatVector([Coords[:,1].min(), Coords[:,1].max()])

# sDomain = r.cbind(vector1, vector2)


# count=0
# for j in range(tr):
    
#     TempData=MouseData[:,j+3]#-MouseData[:,3:].mean(axis=1)
#     grid_z = griddata((x, y), TempData, (grid_x, grid_y), method='linear')

#     interp_TempData = griddata((grid_x.ravel(), grid_y.ravel()), grid_z.ravel(), (x, y), method='linear')
#     interp_TempData=np.where(np.isnan(interp_TempData),0,interp_TempData)
    
#     UpperThreshold= np.percentile(interp_TempData, 75, axis=0)
#     #LowerThreshold= np.percentile(interp_TempData, 25, axis=0)
#     Mask=np.logical_and(interp_TempData>UpperThreshold,np.abs(interp_TempData)>0)
#     MaskAllFrames[:,j]=Mask
    
#     X_temp=Coords[Mask==1,0]

#     X1=FloatVector(X_temp[:])

#     X_temp=Coords[Mask==1,1]

#     X2=FloatVector(X_temp[:])
#     Y_temp=TempData[Mask==1]

#     Y=FloatVector(Y_temp[:])
    
#     df = pd.DataFrame({'X1': X1, 'X2': X2})

#     r_dataframe = pandas2ri.py2rpy(df)
    
#     LKOutput=LK.LatticeKrig(r_dataframe,Y,nlevel = 3,findAwght = True)#,LKinfo='NONE')#,findAwght = True)

#     X1_all=FloatVector(Coords[:,0])
#     X2_all=FloatVector(Coords[:,1])
#     Y_all=FloatVector(TempData)

#     df_all = pd.DataFrame({'X1': X1_all, 'X2': X2_all})

#     r_dataframe_all = pandas2ri.py2rpy(df_all)
#     LKPredAllFrames[:,count]=r.predict(LKOutput,r_dataframe_all).ravel()
#     count=count+1
#     print(count)


In [ ]:
# Set up the lattice krig approach and function to be run with 

import numpy as np
from rpy2.robjects.packages import importr
from rpy2.robjects import FloatVector
from rpy2.robjects import r, pandas2ri
import rpy2.robjects as robjects
import pandas as pd
import scipy
from sklearn.neighbors import DistanceMetric
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.interpolate import griddata

pandas2ri.activate()
LK = importr('LatticeKrig')
tr=MouseData.shape[1]-3
nTR=540
LKPredAllFrames=np.zeros([MouseData.shape[0],nTR])
MaskAllFrames=np.zeros([MouseData.shape[0],nTR])



# Example data

x=MouseData[:,1]
y=MouseData[:,2]


# Define the grid for interpolation
grid_x, grid_y = np.mgrid[x.min():x.max():25j, y.min():y.max():25j]

# Perform interpolation
Coords=MouseData[:,1:3]
vector1 = robjects.FloatVector([Coords[:,0].min(), Coords[:,0].max()])
vector2 = robjects.FloatVector([Coords[:,1].min(), Coords[:,1].max()])

sDomain = r.cbind(vector1, vector2)


count=0
def LKrig(j):
#for j in range(1):
    
    TempData=MouseData[:,j+3]#-MouseData[:,3:].mean(axis=1)
    grid_z = griddata((x, y), TempData, (grid_x, grid_y), method='linear')

    interp_TempData = griddata((grid_x.ravel(), grid_y.ravel()), grid_z.ravel(), (x, y), method='linear')
    interp_TempData=np.where(np.isnan(interp_TempData),0,interp_TempData)
    
    UpperThreshold= np.percentile(interp_TempData, 75, axis=0)
    #LowerThreshold= np.percentile(interp_TempData, 25, axis=0)
    Mask=np.logical_and(interp_TempData>UpperThreshold,np.abs(interp_TempData)>0)
    #MaskAllFrames[:,j]=Mask
    
    X_temp=Coords[Mask==1,0]

    X1=FloatVector(X_temp[:])

    X_temp=Coords[Mask==1,1]

    X2=FloatVector(X_temp[:])
    Y_temp=TempData[Mask==1]

    Y=FloatVector(Y_temp[:])
    
    df = pd.DataFrame({'X1': X1, 'X2': X2})

    r_dataframe = pandas2ri.py2rpy(df)
    
    
    
    LKOutput=LK.LatticeKrig(r_dataframe,Y,nlevel = 3,findAwght = True)
    #LKOutput=LK.LatticeKrig(r_dataframe,Y,nlevel = 3,findAwght = False)

    #LKOutput=LK.LatticeKrig(r_dataframe,Y,LKinfo=LKinfo)#,findAwght = 'TRUE')

    X1_all=FloatVector(Coords[:,0])
    X2_all=FloatVector(Coords[:,1])
    Y_all=FloatVector(TempData)

    df_all = pd.DataFrame({'X1': X1_all, 'X2': X2_all})

    r_dataframe_all = pandas2ri.py2rpy(df_all)
    return r.predict(LKOutput,r_dataframe_all).ravel(), Mask
    
    






In [ ]:
import multiprocess as mp
import time

st=time.time()
with mp.Pool(8) as pool:
    LKPredAllFrames =pool.map(LKrig, np.arange(nTR))
print(time.time()-st)

np.save('MovieLKPred',np.asarray(LKPredAllFrames))

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
LKPredAllFrames=np.load('MovieLKPred.npy')


q=505
plt.scatter(LKPredAllFrames[q,0,:],MouseData[:,q])





In [ ]:
from sklearn.metrics import pairwise_distances_argmin_min
from scipy import stats 
from matplotlib.transforms import Bbox

j=1
DistThr=50

plt.rcParams["font.family"] = "Arial"
TempData=MouseData[:,j+3]

Mask=LKPredAllFrames[j,1,:]
[DistInd,Dist]=pairwise_distances_argmin_min(Coords,Coords[Mask==1])


print(stats.spearmanr(TempData[Mask==0],LKPredAllFrames[j,0,Mask==0]))
print(stats.spearmanr(TempData[Dist>DistThr],LKPredAllFrames[j,0,Dist>DistThr])[0])

fig, axs = plt.subplots(1, 3,figsize=(15,5))

TempBothMasks=((Mask==1)*1+(Dist>DistThr)*-1)




axs[0].scatter(MouseData[:,1],MouseData[:,2],c=TempData,s=0.1,cmap='bwr')
axs[0].set_aspect('equal', 'box')
#axs[1, 0].set_title('Real', fontsize=10)
axs[0].set_xticks([])
axs[0].set_yticks([])
axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)
axs[0].spines['left'].set_visible(False)
axs[0].spines['bottom'].set_visible(False)


axs[1].scatter(MouseData[:,1],MouseData[:,2],c=LKPredAllFrames[j,0,:],s=0.1,cmap='bwr')
axs[1].set_aspect('equal', 'box')
#axs[1, 1].set_title('Projected', fontsize=10)
axs[1].set_xticks([])
axs[1].set_yticks([])
axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)
axs[1].spines['left'].set_visible(False)
axs[1].spines['bottom'].set_visible(False)

axs[2].scatter(TempData[:], LKPredAllFrames[j,0,:],s=0.1,cmap='bwr')
axs[2].set_aspect('equal', 'box')

left, bottom, width, height = (TempData[:].min(), LKPredAllFrames[j,0,:].min(), np.abs(TempData[:].min()), np.abs(LKPredAllFrames[j,0,:].min()))
rect = plt.Rectangle((left, bottom), width, height,facecolor="blue", alpha=0.1)
axs[2].add_patch(rect)

bbox = Bbox.from_bounds(left, bottom, width, height)

#axs[2, 0].set_title('Real versus projected', fontsize=10)

#axs[1, 1].scatter(TempData[Dist>DistThr], LKPredAllFrames[j,0,Dist>DistThr],s=0.1,cmap='bwr')
#axs[2, 1].set_title('Real versus proj Min distance', fontsize=10)
#axs[1, 1].set_box_aspect(1)

plt.show()

fig.savefig('IllustrationMouse5SingleFrame.png',dpi=600)


In [ ]:
np.abs(LKPredAllFrames[j,0,:]).min()

In [ ]:

nTR=540

Corrs25=np.zeros([nTR])
CorrsMinDist25=np.zeros([nTR])

from sklearn.metrics import pairwise_distances_argmin_min

#for j in range(LKPredAllFrames.shape[1]):
for j in range(nTR):
    DistThr=25
    TempData=MouseData[:,j+3]
    LKPred=LKPredAllFrames[j,0,:]    
    Mask=LKPredAllFrames[j,1,:]

    [DistInd,Dist]=pairwise_distances_argmin_min(Coords,Coords[Mask==1])

    Corrs25[j]=stats.spearmanr(TempData[Mask==0],LKPred[Mask==0])[0]
    CorrsMinDist25[j]=stats.spearmanr(TempData[Dist>DistThr],LKPred[Dist>DistThr])[0]



In [ ]:
import seaborn as sns
#plt.hist(CorrsMinDist,bins=50)
plt.rcParams["font.family"] = "Arial"
mean25=np.mean(np.arctanh(CorrsMinDist25))
#mean25=np.mean((CorrsMinDist25))

#mean50=np.mean(np.arctanh(CorrsMinDist))



fig, axes = plt.subplots(1, 1,figsize=(5,5))


#sns.kdeplot(CorrsMinDist25,fill=True)
sns.histplot(CorrsMinDist25, kde=True,bins=100,binrange=[-1,1],ax=axes)
axes.axvline(mean25, color='k', linestyle='--', label="Mean")
axes.axvline(0, color='r', linestyle='--', label="Mean")

axes.set(ylabel=None)


#sns.kdeplot(CorrsMinDist,fill=True)
#sns.histplot(np.arctanh(CorrsMinDist), kde=True,bins=100,binrange=[-2,2],ax=axes[1])
#axes[0].axvline(mean50, color='k', linestyle='--', label="Mean")
#axes[0].axvline(0, color='r', linestyle='--', label="Mean")

fig.savefig('DistAcrossAllFramesMouse25MinDist.png',dpi=600)



In [ ]:
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.graphics.tsaplots import plot_pacf

pacf = plot_pacf(np.arctanh(CorrsMinDist25), lags=25)
ar_model = AutoReg(np.arctanh(CorrsMinDist25), lags=1).fit()
print(ar_model.summary())

pacf = plot_pacf(np.arctanh(CorrsMinDist), lags=25)
ar_model = AutoReg(np.arctanh(CorrsMinDist), lags=1).fit()
print(ar_model.summary())


In [ ]:
from sklearn.metrics import pairwise_distances_argmin_min
import scipy.stats 

j=220
DistThr=50

TempData=MouseData[:,j+3]

Mask=LKPredAllFrames[j][1]
[DistInd,Dist]=pairwise_distances_argmin_min(Coords,Coords[Mask==1])
plt.scatter(MouseData[:,1],MouseData[:,2],c=LKPredAllFrames[j][0],s=0.1)
plt.colorbar()
plt.show()

plt.scatter(MouseData[:,1],MouseData[:,2],c=TempData,s=0.1)
plt.colorbar()
plt.show()

plt.scatter(MouseData[:,1],MouseData[:,2],c=Mask,s=0.1)
plt.colorbar()
plt.show()
plt.scatter(TempData[Mask==0],LKPredAllFrames[j][0][Mask==0],s=0.1)
plt.show()
plt.scatter(MouseData[:,1],MouseData[:,2],c=Dist>DistThr,s=0.1)
plt.colorbar()
plt.show()
plt.scatter(TempData[Dist>DistThr],LKPredAllFrames[j][0][Dist>DistThr],s=0.1)
plt.colorbar()
plt.show()

print(stats.spearmanr(TempData[Mask==0],LKPredAllFrames[j][0][Mask==0]))
print(stats.spearmanr(TempData[Dist>DistThr],LKPredAllFrames[j][0][Dist>DistThr])[0])

In [ ]:


#nFrames=
Corrs=np.zeros([nTR])
CorrsMinDist=np.zeros([nTR])


from sklearn.metrics import pairwise_distances_argmin_min

for j in range(nTR):
    DistThr=50
    TempData=MouseData[:,j+3]
    LKPred=LKPredAllFrames[j][0]    
    Mask=Mask=LKPredAllFrames[j][1]

    [DistInd,Dist]=pairwise_distances_argmin_min(Coords,Coords[Mask==1])

    Corrs[j]=stats.spearmanr(TempData[Mask==0],LKPred[Mask==0])[0]
    CorrsMinDist[j]=stats.spearmanr(TempData[Dist>DistThr],LKPred[Dist>DistThr])[0]



In [ ]:
print(np.mean(Corrs))
print(np.mean(CorrsMinDist))
print(np.median(CorrsMinDist))
plt.hist(CorrsMinDist)
stats.ttest_1samp(CorrsMinDist,0)
print((CorrsMinDist>0).sum())

In [ ]:

nFrames=LKPredAllFrames.shape[1]
nFrames=166
Corrs=np.zeros([nFrames])
CorrsMinDist=np.zeros([nFrames])


from sklearn.metrics import pairwise_distances_argmin_min

#for j in range(LKPredAllFrames.shape[1]):
for j in range(nFrames):
    DistThr=25
    TempData=MouseData[:,j+3]

    Mask=MaskAllFrames[:,j]
    [DistInd,Dist]=pairwise_distances_argmin_min(Coords,Coords[Mask==1])

    Corrs[j]=stats.spearmanr(TempData[Mask==0],LKPredAllFrames[Mask==0,j])[0]
    CorrsMinDist[j]=stats.spearmanr(TempData[Dist>DistThr],LKPredAllFrames[Dist>DistThr,j])[0]



In [ ]:
print(np.mean(CorrsMinDist))
print(np.mean(CorrsMinDist))
plt.hist(CorrsMinDist)